pyomo prac


Ten jobs are to be completed by three workers during the next week. Each worker has a 40-hour work week. The times for the workers to complete the jobs are shown in the table. The values in the cells assume that each job is completed by a single worker; however, jobs can be shared, with completion times being determined proportionally If no entry exists in a particular cell, it means that the corresponding job cannot be performed by the corresponding worker. Set up and solve an LP model that will determine the optimal assignment of workers to jobs. The goal is to minimize the total time required to complete all the jobs.

| Workers \ Tasks | 1  | 2  | 3  | 4  | 5  | 6  | 7  | 8  | 9  | 10 |
|-----------------|----|----|----|----|----|----|----|----|----|----|
| A               | -  | 7  | 3  | -  | -  | 18 | 13 | 6  | -  | 9  |
| B               | 12 | 5  | -  | 12 | 4  | 22 | -  | 17 | 13 | -  |
| C               | 18 | -  | 6  | 8  | 10 | -  | 19 | -  | 8  | 15 |


In [1]:
# imports
import pandas as pd
import pyomo.environ as pe 
import pyomo.opt as po 

In [2]:
# data definition

workers = { 'A', 'B', 'C'}

tasks = set(range(1, 11))

c = {
    ('A',  2):  7,
    ('A',  3):  3,
    ('A',  6): 18,
    ('A',  7): 13,
    ('A',  8):  6,
    ('A', 10):  9,
    ('B',  1): 12,
    ('B',  2):  5,
    ('B',  4): 12,
    ('B',  5):  4,
    ('B',  6): 22,
    ('B',  8): 17,
    ('B',  9): 13,
    ('C',  1): 18,
    ('C',  3):  6,
    ('C',  4):  8,
    ('C',  5): 10,
    ('C',  7): 19,
    ('C',  9):  8,
    ('C', 10): 15,
}

max_hours = 40

Model

Define W 
 as the set of workers and T
 as the sets of tasks. Also, define c_w_t   
 as the number of hours worker w 
 requires to complete task t 
. (Note that we do not explicitly prohibit a worker from completiting as task; rather, we make the cost arbitrarily large if worker w 
 is unable to perform task t 
.) Let x_w_t 
 be the proportion of task t 
 that is completed by worker j 
. Let H 
 be the max number of hours that any single worker may log in a week. We formulate as follows.

\begin{alignat*}{3}
\text{minimize  }  & \sum_{w \in W} \sum_{t \in T} c_{wt} x_{wt} && \\
\text{subject to  }
& \sum_{t \in T} c_{wt} x_{wt} \le H,
&& \qquad \forall w \in W \\
& \sum_{w \in W} x_{wt} = 1
&& \qquad \forall t \in T \\
& 0 \le x_{wt} \le 1,
&& \qquad \forall w \in W, \forall t \in T
\end{alignat*}

In [8]:
# implementation

model = pe.ConcreteModel()

In [9]:
# Sets
model.workers = pe.Set(initialize=workers)
model.tasks = pe.Set(initialize=tasks)

source (type: set).  This WILL potentially lead to nondeterministic behavior
in Pyomo
source (type: set).  This WILL potentially lead to nondeterministic behavior
in Pyomo


In [10]:
# Parameters 
model.c = pe.Param(model.workers, model.tasks, initialize = c, default = 1000)
model.max_hours = pe.Param(initialize = max_hours)

In [12]:
# Variable
model.x = pe.Var(model.workers, model.tasks, domain = pe.Reals, bounds = (0, 1))

In [13]:
# objective function
expr = sum(model.c[w, t] * model.x[w, t] for w in model.workers for t in model.tasks)
model.objective = pe.Objective(sense = pe.minimize, expr = expr)

In [14]:
# Constraints 
model.task_done = pe.ConstraintList()
for t in model.tasks:
    lhs = sum(model.x[w, t] for w in model.workers)
    rhs = 1
    model.task_done.add(lhs == rhs)

In [15]:
model.hour_limit = pe.ConstraintList()
for w in model.workers:
    lhs = sum(model.c[w, t] * model.x[w, t] for t in model.tasks)
    rhs = model.max_hours
    model.hour_limit.add(lhs <= rhs)

In [16]:
# Solving

solver = po.SolverFactory('glpk')
results = solver.solve(model, tee = True)

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\chidu\AppData\Local\Temp\tmpoeyzdhm9.glpk.raw --wglp C:\Users\chidu\AppData\Local\Temp\tmpm8f4y8ot.glpk.glp
 --cpxlp C:\Users\chidu\AppData\Local\Temp\tmpunirq_qp.pyomo.lp
Reading problem data from 'C:\Users\chidu\AppData\Local\Temp\tmpunirq_qp.pyomo.lp'...
13 rows, 30 columns, 60 non-zeros
168 lines were read
Writing problem data to 'C:\Users\chidu\AppData\Local\Temp\tmpm8f4y8ot.glpk.glp'...
179 lines were written
GLPK Simplex Optimizer 5.0
13 rows, 30 columns, 60 non-zeros
Preprocessing...
13 rows, 30 columns, 60 non-zeros
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+03  ratio =  1.000e+03
GM: min|aij| =  2.383e-01  max|aij| =  4.197e+00  ratio =  1.761e+01
EQ: min|aij| =  5.678e-02  max|aij| =  1.000e+00  ratio =  1.761e+01
Constructing initial basis...
Size of triangular part is 13
      0: obj =   3.085000000e+03 inf =   1.189e+01 (1)
     10: obj =   9.545454545e+01 inf =   2.

In [17]:
df = pd.DataFrame(index = pd.MultiIndex.from_tuples(model.x, names = ['w', 't']))
df['x'] = [pe.value(model.x[key]) for key in model.x]
df['c'] = [model.c[key] for key in df.index]

In [18]:
(df[ 'c'] * df['x']).unstack('t')

t,1,2,3,4,5,6,7,8,9,10
w,,,,,,,,,,
A,0.0,0.0,3.000000e+00,0.0,0.0,9.0,13.0,6.0,0.0,9.0
B,12.0,5.0,0.000000e+00,0.0,4.0,11.0,0.0,0.0,0.0,0.0
C,0.0,0.0,-7.244857e-16,8.0,0.0,0.0,0.0,0.0,8.0,0.0


In [20]:
(df['c'] * df['x']).groupby('w').sum().to_frame()

,0
w,
A,40.0
B,32.0
C,16.0


In [21]:
df['x'].groupby('t').sum().to_frame().Tx    

t,1,2,3,4,5,6,7,8,9,10
x,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
